In [ ]:
#문서 전처리 함수
def preprocess(df_add):
    doc_list = []
    for doc in df_add:
        #구두점 제거
        doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()

        #숫자 제거
        doc2 = "".join([i for i in doc1 if not i.isdigit()])

        #월 제거
        month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
                 'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   

        doc3 = " ".join([i for i in doc2.split() if i not in month])
        doc_list.append(doc3)
    
    return doc_list

#문서 바이그램 단위로 나누는 함수 + nltk 제공하는 불용어 제거
def bigram(doc_list):
    n_gram_range = (2, 2)
    stop_words = "english"

    candidate_list = []
    for doc in doc_list:
        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
        candidate = count.get_feature_names_out()
        candidate_list.append(candidate)
    
    return candidate_list

#전처리 문서와 바이그램 embedding 후 유사도 높은 20개 키워드 추출 함수
def bigram_embedding1(doc_list, candidate_list):
    bigram_keywords1 = []
    top_n = 20

    for i in range(len(doc_list)):
        doc_embeddings = model.encode([doc_list[i]])
        candidate_embeddings = model.encode(candidate_list[i])
        distances = cosine_similarity(doc_embeddings, candidate_embeddings)
        bigram_keywords1.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])
        
    return bigram_keywords1

#원본 문서(전처리 X)와 바이그램 embedding 후 유사도 높은 20개 키워드 추출 함수
def bigram_embedding2(doc_list, candidate_list):
    bigram_keywords2 = []
    top_n = 20

    for i in range(len(doc_list)):
        doc_embeddings = model.encode([df_add[i]])
        candidate_embeddings = model.encode(candidate_list[i])
        distances = cosine_similarity(doc_embeddings, candidate_embeddings)
        bigram_keywords2.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])
        
    return bigram_keywords2

#추출 키워드 embedding 함수
def keyword_embedding(bigram_keywords):
    bigram_embeddings = []
    for i in range(len(doc_list)):
        bigram_embedding = []
        for keyword in bigram_keywords[i]:
            bigram_embedding.append(model.encode(keyword))
        bigram_embeddings.append(bigram_embedding)
    
    return bigram_embeddings

#선정 키워드(단어) 불러오기
def get_keyword1():
    df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
    keyword = list(df_keyword["번역"])
    keyword_embeddings = []

    for ele in keyword:
        keyword_embeddings.append(model.encode(ele))
    
    return keyword, keyword_embeddings

#선정 키워드(단어) 불러오기
def get_keyword2():
    df_keyword = pd.read_csv("임시.csv", index_col = False)
    keyword = list(df_keyword["description"])
    keyword_embeddings = []

    for ele in keyword:
        keyword_embeddings.append(model.encode(ele))
    
    return keyword, keyword_embeddings

#추출 키워드와 선정 키워드 유사도 비교
def similarity_test(bigram_embeddings, keyword_embeddings, bigram_keywords, keyword):
    bigram_result = []
    keyword_result = []
    cosine_result = []
    for index, bigram in enumerate(bigram_embeddings): #2번 반복(총 2개 문서)

        b_result = []
        k_result = []
        c_result = []

        for i in range(len(bigram)): # 20번 반복(top 20)

            for j in range(len(keyword_embeddings)): #102번 반복(keyword 개수)

                distances = cosine_similarity([bigram[i]],[keyword_embeddings[j]]) #유사도 비교

                if distances[0][0] > 0.48:

                    b_result.append(bigram_keywords[index][i])
                    k_result.append(keyword[j])
                    c_result.append(str(round(float(distances),3)))

        bigram_result.append(b_result)
        keyword_result.append(k_result)
        cosine_result.append(c_result)
    return bigram_result, keyword_result, cosine_result

#유사도 높은 순으로 df 만들기
def make_df(bigram_result, keyword_result, cosine_result):
    df_final = pd.DataFrame()

    bigram_list = []
    keyword_list = []
    distance = []

    for i in range(len(df_add)):
        B, K, D = [], [], []
        b_result,k_result, d_result = "", "", ""
        df_check = pd.DataFrame()
        df_check['bigram'] = pd.Series(bigram_result[i])
        df_check['keyword'] = pd.Series(keyword_result[i])
        df_check['distance'] = pd.Series(cosine_result[i])

        df_check = df_check.sort_values(by="distance", ascending=False)

        B = df_check['bigram'].tolist()
        K = df_check['keyword'].tolist()
        D = df_check['distance'].tolist()

        for b in B:
            b_result = b_result + "/" + b
        for k in K:
            k_result = k_result + "/" + k
        for d in D:
            d_result = d_result + "/" + d


        bigram_list.append(b_result[1:])
        keyword_list.append(k_result[1:])
        distance.append(d_result[1:])

        del df_check

    df_final['bigram'] = bigram_list
    df_final['keyword'] = keyword_list
    df_final['distance'] = distance

    return df_final


In [1]:
#문서 전처리 함수
def preprocess(df_add):
    doc_list = []
    for doc in df_add:
        #구두점 제거
        doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()

        #숫자 제거
        doc2 = "".join([i for i in doc1 if not i.isdigit()])

        #월 제거
        month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
                 'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   

        doc3 = " ".join([i for i in doc2.split() if i not in month])
        doc_list.append(doc3)
    
    return doc_list

In [2]:
#문서 바이그램 단위로 나누는 함수 + nltk 제공하는 불용어 제거
def bigram(doc_list):
    n_gram_range = (2, 2)
    stop_words = "english"

    candidate_list = []
    for doc in doc_list:
        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
        candidate = count.get_feature_names_out()
        candidate_list.append(candidate)
    
    return candidate_list

In [3]:
#전처리 문서와 바이그램 embedding 후 유사도 높은 20개 키워드 추출 함수
def bigram_embedding1(doc_list, candidate_list):
#     model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    bigram_keywords1 = []
    top_n = 20

    for i in range(len(doc_list)):
        doc_embeddings = model.encode([doc_list[i]])
        candidate_embeddings = model.encode(candidate_list[i])
        distances = cosine_similarity(doc_embeddings, candidate_embeddings)
        bigram_keywords1.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])
        
    return bigram_keywords1

In [4]:
#원본 문서(전처리 X)와 바이그램 embedding 후 유사도 높은 20개 키워드 추출 함수
def bigram_embedding2(doc_list, candidate_list):
#     model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    bigram_keywords2 = []
    top_n = 20

    for i in range(len(doc_list)):
        doc_embeddings = model.encode([df_add[i]])
        candidate_embeddings = model.encode(candidate_list[i])
        distances = cosine_similarity(doc_embeddings, candidate_embeddings)
        bigram_keywords2.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])
        
    return bigram_keywords2

In [5]:
#추출 키워드 embedding 함수
def keyword_embedding(bigram_keywords):
    bigram_embeddings = []
    for i in range(len(doc_list)):
        bigram_embedding = []
        for keyword in bigram_keywords[i]:
            bigram_embedding.append(model.encode(keyword))
        bigram_embeddings.append(bigram_embedding)
    
    return bigram_embeddings

In [6]:
#선정 키워드(단어) 불러오기
def get_keyword1():
    df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
    keyword = list(df_keyword["번역"])
    keyword_embeddings = []

    for ele in keyword:
        keyword_embeddings.append(model.encode(ele))
    
    return keyword, keyword_embeddings

In [7]:
#선정 키워드(단어) 불러오기
def get_keyword2():
    df_keyword = pd.read_csv("임시.csv", index_col = False)
    keyword = list(df_keyword["description"])
    keyword_embeddings = []

    for ele in keyword:
        keyword_embeddings.append(model.encode(ele))
    
    return keyword, keyword_embeddings

In [17]:
#추출 키워드와 선정 키워드 유사도 비교
def similarity_test(bigram_embeddings, keyword_embeddings, bigram_keywords, keyword):
    bigram_result = []
    keyword_result = []
    cosine_result = []
    for index, bigram in enumerate(bigram_embeddings): #2번 반복(총 2개 문서)

        b_result = []
        k_result = []
        c_result = []

        for i in range(len(bigram)): # 20번 반복(top 20)

            for j in range(len(keyword_embeddings)): #102번 반복(keyword 개수)

                distances = cosine_similarity([bigram[i]],[keyword_embeddings[j]]) #유사도 비교

                if distances[0][0] > 0.48:

                    b_result.append(bigram_keywords[index][i])
                    k_result.append(keyword[j])
                    c_result.append(str(round(float(distances),3)))

        bigram_result.append(b_result)
        keyword_result.append(k_result)
        cosine_result.append(c_result)
    return bigram_result, keyword_result, cosine_result

In [9]:
#유사도 높은 순으로 df 만들기
def make_df(bigram_result, keyword_result, cosine_result):
    df_final = pd.DataFrame()

    bigram_list = []
    keyword_list = []
    distance = []

    for i in range(len(df_add)):
        B, K, D = [], [], []
        b_result,k_result, d_result = "", "", ""
        df_check = pd.DataFrame()
        df_check['bigram'] = pd.Series(bigram_result[i])
        df_check['keyword'] = pd.Series(keyword_result[i])
        df_check['distance'] = pd.Series(cosine_result[i])

        df_check = df_check.sort_values(by="distance", ascending=False)

        B = df_check['bigram'].tolist()
        K = df_check['keyword'].tolist()
        D = df_check['distance'].tolist()

        for b in B:
            b_result = b_result + "/" + b
        for k in K:
            k_result = k_result + "/" + k
        for d in D:
            d_result = d_result + "/" + d


        bigram_list.append(b_result[1:])
        keyword_list.append(k_result[1:])
        distance.append(d_result[1:])

        del df_check

    df_final['bigram'] = bigram_list
    df_final['keyword'] = keyword_list
    df_final['distance'] = distance

    return df_final


In [10]:
#테스트 진행 위한 준비
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import itertools
import string
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [11]:
df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i].lower()+""+df_text[i])

In [12]:
#테스트 진행

#문서 전처리
doc_list = preprocess(df_add)

#문서에서 바이그램 추출
candidate_list = bigram(doc_list)

#전처리 문서/원본 문서와 바이그램 유사도 비교 후 top 20 바이그램 선정
bigram_keywords1 = bigram_embedding1(doc_list, candidate_list)
bigram_keywords2 = bigram_embedding2(doc_list, candidate_list)

#top 20 바이그램 embedding 진행
bigram_embeddings1 = keyword_embedding(bigram_keywords1)
bigram_embeddings2 = keyword_embedding(bigram_keywords2)

#선정 키워드(단어/설명) 불러오기
keyword1, keyword_embeddings1 = get_keyword1()
keyword2, keyword_embeddings2 = get_keyword2()

In [18]:
bigram_result, keyword_result, cosine_result = similarity_test(bigram_embeddings1, keyword_embeddings1, bigram_keywords1, keyword1)

In [14]:
# make_df(bigram_result, keyword_result, cosine_result)

In [22]:
df1 = pd.read_csv("호주_세관_유사도 비교 위한 요약.csv", index_col = False)
df2 = make_df(bigram_result, keyword_result, cosine_result)

df2 = pd.concat([df1,df2],axis=1)
df2

C:\Users\water\AppData\Local\Temp\ipykernel_13440\3787758834.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['bigram'] = pd.Series(bigram_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13440\3787758834.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['keyword'] = pd.Series(keyword_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13440\3787758834.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_check['distance'] = pd.Series(cosine_result[i])
C:\Users\water\AppData\Local\Temp\ipykernel_13440\3787758834.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' 

,date,link,title,summary,bigram,keyword,distance
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,"주류, 맥주, 연료 제품을 포함한 특정 소비재 관련 새로운 관세율 명시",,,
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,"전기 차, 하이브리드 차량, 연료 효율 높은 자동차에 대한 대한 관세 철폐",vehicles australian/efficient vehicles/car tax...,car/car/car/car/car/Automotive Parts/car,0.592/0.498/0.495/0.493/0.489/0.488/0.483
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,브로커 면허 신청,,,
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,우크라이나로부터의 상품에 대한 임시 관세 감면,product australia/importation goods,agricultural products/agricultural products,0.503/0.488
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,브로커 면허 신청,,,
5,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,Cargo reporting requirements for international...,국제 우편 및 특정 업무 문서 관련 통지,,,
6,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,New timeframes for the disposal of unentered a...,미입고 및 폐기물의 폐기 관련 통지,imported goods/clear goods/imported goods,agricultural products/Cleansing supplies/dairy...,0.547/0.53/0.495
7,2022-06-29,https://www.abf.gov.au/help-and-support-subsit...,Tariff Concession – Item 57 – Revised,관세 변경 통지 - 의약품,containers medicaments/containers medicaments/...,Medical container/Medical container parts/Clea...,0.825/0.739/0.662/0.628/0.615/0.604/0.597/0.58...
8,2022-06-28,https://www.abf.gov.au/help-and-support-subsit...,Commencement of phase 2 of the Recycling and W...,재활용 및 폐기물 감소 - 플라스틱,plastic declare/plastic rules/waste plastic/wa...,Plastic/Plastic/Plastic/Plastic/Plastic/Plasti...,0.807/0.743/0.74/0.737/0.721/0.72/0.714/0.689/...
9,2022-06-27,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,브로커 면허 신청,,,


In [23]:
df2.to_csv('호주_세관 비교.csv', mode='w', encoding='utf-8-sig', index = False)